This notebook contains first descriptive plots to get the intuition about the inputs to the simulation.

The structure is the following:
* reference matrices
* mask wearing data (total, per age group)
* average contact numbers (total, per age group)

In [ ]:
import cycler
from datetime import datetime, timedelta
import json
import os
import sys
from warnings import filterwarnings

# from matplotlib.colors import LogNorm
import json
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append("../")
from src.dataloader import PROJECT_PATH
from src.simulation import Simulation

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

filterwarnings('default')
def_cycler = plt.rcParams['axes.prop_cycle']
os.makedirs(os.path.join(PROJECT_PATH, "plots"), exist_ok=True)
%matplotlib inline

In [ ]:
plt.rcParams["font.size"] = 18
plt.rcParams["figure.facecolor"] = "white"

# Different reference matrix options

## Online reference matrix

In this version, we estimate reference proxy contact numbers before COVID from the responses of the online survey
referring to the reference period ($OR$ - online reference proxy). We use responses of users from mid-April until
the end of June (`2020/04/14-2020/06/30`), because people tend to underestimate their reference proxy contacts
as pre-COVID times are getting further.

Unfortunately, we don't have online reference proxy contact numbers for children ($OR_{k}$ -
online reference proxy kids), and the representative proxy contact numbers ($RR$ - representative reference proxy)
are consistently higher than in the online survey (so we cannot automatically fill the missing children columns from
the representative survey).

Therefore, we try to infer online reference kid proxy contacts ($OR_{k}$) by multiplying their September online actual
proxy contacts ($OA_{k}$ - online actual proxy kids) with the fraction of the representative reference proxy ($RR_{k}$
- representative reference proxy kids) and representative actual proxy contacts ($RA_{k}$ -
representative actual proxy kids). We use September data for the online actual kid proxy contacts,
because schools were open.
Maybe instead of the above text, this equation is somewhat more readable, for every matrix element for kids:

$$OR_{k} = OA_{k,\ \mathrm{Sept}}\frac{RR_{k,\ \mathrm{May}}}{RA_{k,\ \mathrm{Sept}}}$$

Here are the multipliers visually for the first 3 columns (age<18) for weekdays and weekends:
![](./img/online_reference_estimation_1.png)

**Contacts between 0-2 and 7 age groups are completely missing from $RR$, it cancels out every other values.**

For both the estimation of $OR_{a}$ (adults) and $OR_{k}$ (kids), we have separate matrices for weekdays and weekends.
Here are the estimated weekend-weekday matrices for the kids, then for the adults:

![](./img/online_reference_estimation_2.png)

**There are only 8 days in the dataset from which we estimate weekend kid contacts. It is clearly not enough,
many of the cells don't contain any values!**
![](./img/online_reference_estimation_3.png)


We can observe that the second age group consists of both kids (15-17) and adults (18-25),
so we have to mix that column according to the proportions within
the age group ($N_{15-17} = 287712$, $N_{18-25}=1 371 985$, source https://www.ksh.hu/interaktiv/korfak/orszag.html).

$$OR^{2nd\ col} = \frac{N_{15-17}}{N_{15-25}}\cdot OR_{k}^{2nd\ col} +
\left(1-\frac{N_{15-17}}{N_{15-25}}\right)\cdot OR_{a}^{2nd\ col}$$

After having calculated / estimated, and put together the two matrix parts $OR_k$ and $OR_a$, we get the final $OR$ as:

$$OR = \frac{5\cdot OR^{weekday} + 2\cdot OR^{weekend}}{7}$$.

![](./img/online_reference_estimation_4.png)

The reference matrix $R$ is then the sum of $OR$, and the family "background" matrix $F$ for the same time period
(`2020/04/14-2020/06/30`):

$$R = OR + F.$$

![](./img/online_reference_estimation_5.png)

The resulting matrix file is located at `contact_matrix/reference_online.csv` in the repo.
The then select a literature $R_0$ value corresponding to this contact matrix.

## Representative reference matrix

In this version, we use the estimated reference contact numbers from the representative survey.
The problem is that the representative survey gives consistently larger contact numbers than the online survey,
thus, it might lead to an underestimation of $R_0$ because of the unrealistic drop in the average contact numbers with
respect to the reference period.

The matrix, the previous online reference matrix, and the difference between the two:

![](./img/representative_reference_estimation_1.png)

The representative reference matrix can be found at `contact_matrix/reference_representative.csv`.
We then select a literature $R_0$ value corresponding to this contact matrix.

## Using a selected timeframe as the reference period

Here, we try to pinpoint a selected timeframe, where we use the $OA+F$ (online actual + family) contact matrix
as a reference matrix $TR$ (temporal reference), and the we get the corresponding $R_0$ calculated from the
official infection data to calculate the transmission rate $\beta$.

The problem here are the initial values in the compartments of a model, e.g. latent infections.
The assumption that the whole population is susceptible at this timepoint is not a good IC.

We tried with selecting one week between `2020/08/30` and `2020/09/06`. The selection is done in the `Simulation` class,
but I show this default selection compared to $OR$ and $RR$, the previous reference matrices, below.
![](./img/temporal_reference_estimation_1.png)
![](./img/temporal_reference_estimation_2.png)


# Masking percentage

The next figure shows the global masking percentage extracted from the online answers, created using
the same weighting scheme as for the contact matrices.

In [ ]:
plt.rcParams["font.size"] = 18
plt.rcParams["figure.facecolor"] = "white"

In [ ]:
df = pd.DataFrame(
    json.load(open(
        os.path.join(PROJECT_PATH, 'contact_matrix/dynmatrix_step_1d_window_7d_v15_kid_masked_all.json')))
)

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(30, 10))
plt.plot([datetime.fromtimestamp(t) for t in df.start_ts],
         df.avg_masking * 100,
         'k-', lw=3)
plt.grid()
plt.ylim(0, 100)
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
plt.ylabel("Percentage")
plt.xlabel("Date")
plt.title("Mask wearing percentage from online survey", pad=20)
plt.savefig(os.path.join(PROJECT_PATH, "plots/masking_1.png"), density=300, bbox_inches="tight")

This is broken down to age groups. For kids, we have no answers for mask percentage, for age group 7,
the eldest, the data is very-very noisy.

In [ ]:
n = 7
color = plt.cm.viridis(np.linspace(0, 1, n))[:-2][::-1]
plt.rcParams['axes.prop_cycle'] = cycler.cycler('color', color)

In [ ]:
plt.figure(figsize=(30, 10))
for i in range(2, 7):
    plt.plot([datetime.fromtimestamp(t)
              for t in df.start_ts], df.avg_masking_grouped.map(lambda d: d[str(float(i))]) * 100,
             label="age_group " + str(float(i)), lw=3)
plt.grid()
plt.legend()
plt.ylim(0, 100)
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
plt.ylabel("Percentage")
plt.xlabel("Date")
plt.title("Mask wearing percentage from online survey by age group", pad=20)
plt.savefig(os.path.join(PROJECT_PATH, "plots/mask_2.png"), density=300, bbox_inches="tight")

An artificially smoothed version with a 10-day sliding window average.

In [ ]:
plt.figure(figsize=(30, 10))

N = 11

for i in range(2,7):
    plt.plot(
        [datetime.fromtimestamp(t) for t in df.start_ts][5:][:-5],
        np.convolve(df.avg_masking_grouped.map(lambda d: d[str(float(i))]) * 100, np.ones(N) / N, mode='valid'),
        label="age_group " + str(float(i)),
        lw=3)
plt.grid()
plt.legend()
plt.ylim(0, 100)
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
plt.ylabel("Percentage")
plt.xlabel("Date")
plt.title("Mask wearing percentage from online survey by age group (smoothed)", pad=20)
plt.savefig(os.path.join(PROJECT_PATH, "plots/masking_2.png"), density=300, bbox_inches="tight")

# Contact numbers

In [ ]:
plt.rcParams["font.size"]=30

In [ ]:
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

In [ ]:
df = pd.DataFrame(
    json.load(
        open(os.path.join(PROJECT_PATH, 'contact_matrix/dynmatrix_step_1d_window_7d_v15_kid_masked_all.json')))
)
mask = [datetime.fromtimestamp(t)<datetime(2021,1,1) for t in df.start_ts]


fig, ax1 = plt.subplots(1,1,figsize=(30,10))
ax2 = ax1.twinx()

ax1.plot([datetime.fromtimestamp(t) for t in df[mask].start_ts],
         df[mask].avg_actual_outside_proxy + df[mask].avg_family,
         'k-', lw=3)
ax1.grid()
ax1.set_ylabel("Avg. number of contacts per day",labelpad=30)


sim_1 = Simulation(contact_data_json = 'dynmatrix_step_1d_window_7d_v15_kid_reduced_all.json')
# data from the webpage of Ferenci Tamas
mask = ([t<datetime(2021,1,1) for t in sim_1.data.reference_r_eff_data.datetime]) & \
       (sim_1.data.reference_r_eff_data["method"]=="Cori")

# casenum
N = 3
ax2.plot(sim_1.data.reference_r_eff_data[mask]["datetime"],
         np.convolve(sim_1.data.reference_r_eff_data[mask]["casenum"],
                     np.ones(2 * N + 1) / (2 * N + 1), mode='same'),
         'r--', lw=4, label="number of pos. cases")
ax2.set_ylabel("Number of positive tests",color="r",labelpad=30)


# Define the date format
date_form = DateFormatter("%y %b")
ax2.xaxis.set_major_formatter(date_form)

# plt.savefig(os.path.join(PROJECT_PATH, "plots/contacnum_1.png"), density=300, bbox_inches="tight")

In [ ]:
n = 10
color = plt.cm.viridis(np.linspace(0, 1, n))[:-2][::-1]
plt.rcParams['axes.prop_cycle'] = cycler.cycler('color', color)

In [ ]:
plt.figure(figsize=(30, 10))

N = 3

for i in range(2,7):
    plt.plot(
        [datetime.fromtimestamp(t) for t in df.start_ts][N:][:-N],
        np.convolve(df.avg_actual_outside_proxy_grouped.map(lambda d: d.get(str(float(i)), 0)),
                    np.ones(2 * N + 1) / (2 * N + 1), mode='valid')+\
        np.convolve(df.avg_family_grouped.map(lambda d: d.get(str(float(i)), 0)),
                    np.ones(2 * N + 1) / (2 * N + 1), mode='valid'),
        label="age_group " + str(float(i)),
        lw=3)
plt.grid()
plt.legend()
plt.ylabel("Outside + Family")
plt.xlabel("Date")
plt.ylim(0, 13)
plt.title("Proxy+ family contacts from online survey by age group (smoothed)", pad=20)
plt.savefig(os.path.join(PROJECT_PATH, "plots/contactnum_2.png"),
            density=300,
            bbox_inches="tight")

# Results of simulations

## Globally reduced large matrix element

In [ ]:
plt.rcParams['axes.prop_cycle'] = def_cycler

In [ ]:
sim_1 = Simulation(contact_data_json = 'dynmatrix_step_1d_window_7d_v15_kid_reduced_all.json')

# Call simulate() method to get r_eff values for the contact matrices
sim_1.simulate()

In [ ]:
sim_2 = Simulation(contact_data_json = 'dynmatrix_step_1d_window_7d_v15_kid_reduced_all.json')

# Call simulate() method to get r_eff values for the contact matrices
sim_2.simulate()

In [ ]:
sim_1.data._contact_data_json

In [ ]:
sim_2.data._contact_data_json

In [ ]:
# Calculate datetime from simulation point timestamps for plotting purposes
t1 = [datetime.fromtimestamp(t) for t in sim_1.timestamps]
t2 = [datetime.fromtimestamp(t) for t in sim_2.timestamps]

In [ ]:
# data from the webpage of Ferenci Tamas
df = sim_1.data.reference_r_eff_data

In [ ]:
# source https://www.ksh.hu/docs/hun/xstadat/xstadat_evkozi/e_met001.html
idojaras = pd.read_excel(os.path.join(PROJECT_PATH, "data/weather.xls"), header=[0, 1, 2], index_col=[0, 1])
honapok = {'J':1, 'F':2, 'M':3, 'Á':4, 'Mj':5, 'Jú':6, 'Jl':7, 'Au':8, 'Sz':9, 'O':10, 'N':11, 'D':12, 'J–D':0}
idojaras.index = idojaras.index.map(lambda t: str(int(t[0])) + '-' + str(honapok.get(t[1])).zfill(2))
idojaras.columns = ['temp', 'temp_max', 'temp_min', 'prec_days', 'prec_mm', 'sunny_hours', 'windy_days']
idojaras = idojaras[idojaras.index.map(lambda s: '00' not in s and ('2020' in s or '2021' in s))]
idojaras["date"] = idojaras.index.map(lambda s: datetime.strptime(s, '%Y-%m'))

In [ ]:
%%writefile ../contact_matrix/repr_sum.csv
03/20,28.182647274668152
04/20,5.668536927079893
05/20,6.15521176587219
06/20,8.021268253924598
08/20,7.008615225024456
09/20,8.495211075908282
10/20,9.246222571227818
11/20,5.92951409139812
12/20,7.243891928674314
01/21,7.387639109608567
02/21,6.208721879990596

In [ ]:
!ls ../contact_matrix/repr_sum.csv

In [ ]:
# representative contact numbers
rdf = pd.read_csv(os.path.join(PROJECT_PATH, 'contact_matrix/repr_sum.csv'),sep=",",header=None)
rdf["date"] = rdf[0].map(lambda s: datetime(year=int('20'+s[-2:]),month=int(s[0:2]),day=1)+timedelta(days=30))
rdf.drop(0,axis=1,inplace=True)
rdf.set_index("date",inplace=True)
rdf = rdf.T
rdf["pos"] = (rdf.index-1)%8

In [ ]:
rdf

In [ ]:
plt.figure(figsize=(30,10))

# simulated R-values
plt.plot(t1, sim_1.r_eff_plot, label="new", color="k", lw=4)
plt.plot(t2, sim_2.r_eff_plot, label="old", color="lightgrey", lw=4)

# average contact numbers
plt.plot(
    [datetime.fromtimestamp(t) for t in sim_1.data.contact_num_data["start"]], 
    sim_1.data.contact_num_data["outside"] + sim_1.data.contact_num_data["family"],
    '--',
    label="new contactnum")

plt.plot(
    [datetime.fromtimestamp(t) for t in sim_2.data.contact_num_data["start"]], 
    sim_2.data.contact_num_data["outside"] + sim_2.data.contact_num_data["family"],
    '--',
    label="old contactnum")


# representative contact numbers

plt.step(rdf.groupby("pos").sum().mean().index,rdf.groupby("pos").sum().mean(),'--',
         lw=3, c="darkblue", label="representative contactnum")

# masks
plt.fill_between(
    [datetime.fromtimestamp(t) for t in sim_1.data.contact_num_data["start"]], 
    sim_1.data.contact_num_data['mask_percentage'].map(float),
    alpha=0.2,
    label="mask percentage")

# reference R-values from Tamas Ferenci
for m in ["Cori"]:
    sdf = df[df["method"] == m]
    # positivity rate
    plt.scatter(sdf["datetime"],sdf["r_eff"],
                lw=2, c=sdf["pos"], cmap="Oranges_r",
                vmin=0, vmax=20, label="Ferenci Tamas R_0")
    # errorbar
#     plt.fill_between(sdf["datetime"] - timedelta(days=0), sdf["ci_lower"], sdf["ci_upper"],
#                      color='orange', alpha=0.2)
    # casenum
#     N = 3
#     plt.plot(sdf["datetime"],np.convolve(sdf["casenum"] / 1000, np.ones(2 * N + 1) / (2 * N + 1), mode='same'),
#              'r-', lw=4, alpha=0.3, label="Number of pos. cases")
    
plt.colorbar(label="Test positivity %")
    
# reference R_0 period
d1,d2 =  sim_1.baseline_cm_date
plt.fill_betweenx(
    np.linspace(0, 6, 100),
    datetime.strptime(d1,'%Y-%m-%d'), 
    datetime.strptime(d2,'%Y-%m-%d'),
    color='r',
    alpha=0.2,
    label="reference period")

# 

# formatting
plt.xlabel("Date")
plt.ylabel("R_eff")
plt.ylim(0,12)
plt.grid()
plt.legend(ncol=4)


# # temperature
# ax2 = plt.gca().twinx()
# ax2.step(idojaras["date"].tolist()[2:] + [idojaras["date"].iloc[-1]],
#          idojaras["temp"].tolist()[2:] + [idojaras["temp"].iloc[-1]],
#          lw=2, c="purple", label="temperature")
# ax2.set_ylabel(r"Avg. temperature $(^\circ C)$")
# ax2.legend()


In [ ]:
plt.figure(figsize=(30,10))

# simulated R-values
plt.plot(t1, sim_1.r_eff_plot, color="lightgrey")
plt.plot(t2, sim_2.r_eff_plot, label="online R_0", color="black", lw=4)

# average contact numbers
plt.plot(
    [datetime.fromtimestamp(t) for t in sim_1.data.contact_num_data["start"]], 
    sim_1.data.contact_num_data["outside"] + sim_1.data.contact_num_data["family"],
    '--',
    label="online contactnum")

# masks
plt.fill_between(
    [datetime.fromtimestamp(t) for t in sim_1.data.contact_num_data["start"]], 
    sim_1.data.contact_num_data['mask_percentage'].map(float),
    alpha=0.2,
    label="mask percentage")

# reference R-values from Tamas Ferenci
for m in df["method"].unique()[[1]]:
    sdf = df[df["method"] == m]
    # positivity rate
    plt.scatter(sdf["datetime"] - timedelta(days=0), sdf["r0"],
                lw=2, c=sdf["pos"], cmap="Oranges_r",
                vmin=0, vmax=20, label="Ferenci Tamas R_0")
    # errorbar
    plt.fill_between(sdf["datetime"] - timedelta(days=0), sdf["ci_lower"], sdf["ci_upper"],
                     color='orange', alpha=0.2)
    # casenum
    N = 3
    plt.plot(sdf["datetime"],np.convolve(sdf["casenum"] / 1000, np.ones(2 * N + 1) / (2 * N + 1), mode='same'),
             'r-', lw=4, alpha=0.3, label="number of pos. cases")
    
plt.colorbar(label="Test positivity %")
    
# reference R_0 period
d1,d2 =  sim_2.baseline_cm_date
plt.fill_betweenx(
    np.linspace(0, 6, 100),
    datetime.strptime(d1,'%Y-%m-%d'), 
    datetime.strptime(d2,'%Y-%m-%d'),
    color='r',
    alpha=0.2,
    label="reference period"
)

# 

# formatting
plt.xlabel("Date")
plt.ylabel("R_eff")
plt.ylim(0,8)
plt.grid()
plt.legend()


# temperature
ax2 = plt.gca().twinx()
ax2.step(idojaras["date"].tolist()[2:] + [idojaras["date"].iloc[-1]],
         idojaras["temp"].tolist()[2:] + [idojaras["temp"].iloc[-1]],
         lw=2, c="purple", label="temperature")
ax2.set_ylabel(r"Avg. temperature $(^\circ C)$")
ax2.legend()

In [ ]:
!jupyter nbconvert figures.ipynb --no-input --no-prompt